In [1]:
# ============================================================
# v7_final_debugsafe.py
# 3090(24GB) 안정판: 정렬/정규화/진단 강화 + TTA + OCR + Auto-weight
# - Stratified 5-Fold, 공통 split 고정
# - EfficientNet-B3(380), ViT-B/16(384)
# - Mild Aug, Mixup, LabelSmoothing, AMP(fp16), channels_last
# - ViT grad-checkpointing, 자동 OOM 배치 축소
# - OOF/Test probs npz(float32) 저장 + 과거 npy 호환 로더
# - OCR OOF/Test 안전 정렬 + 확률 재정규화
# - 앙상블 전/후 진단(상관, self-F1, KL, NaN, 합계 등)
# - 제출 전 OOF 기반 Expected F1 계산
# ============================================================

import os, glob, json, random, math, re, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import cv2, timm, torch, albumentations as A
import torch.nn as nn
from PIL import Image
from tqdm import tqdm
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader

# ----------------------------
# Repro & Device
# ----------------------------
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
if hasattr(torch, "set_float32_matmul_precision"):
    torch.set_float32_matmul_precision("high")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} ({torch.cuda.get_device_name(0) if device.type=='cuda' else 'CPU'})")



Using cuda (NVIDIA GeForce RTX 3090)


In [2]:
# ----------------------------
# Paths (v6)
# ----------------------------
BASE = "/data/ephemeral/home/data"
TRAIN_IMG_DIR = f"{BASE}/processed/stage0_6_train_v6/"
TEST_IMG_DIR  = f"{BASE}/processed/stage0_6_test_v6/"
TRAIN_META    = f"{BASE}/meta_stage0_6_train_v6.csv"
TRAIN_CSV     = f"{BASE}/raw/train.csv"
SUB_CSV       = f"{BASE}/raw/sample_submission.csv"

# OCR 확률
OCR_TEST_PROBS_CSV  = f"{BASE}/interim/ocr_test_probs.csv"
OCR_VALID_PROBS_DIR = f"{BASE}/interim/ocr_valid_probs"

os.makedirs("./models", exist_ok=True)
os.makedirs("./oofs", exist_ok=True)
os.makedirs("./test_probs", exist_ok=True)
os.makedirs("./debug", exist_ok=True)

# ----------------------------
# Config
# ----------------------------
NUM_CLASSES = 17
NFOLDS = 5
EPOCHS = 30
INIT_BATCH_SIZE = 16      # RTX 3090(24GB) 기준 안전 시작값
NUM_WORKERS = 6
LR = 1e-4
WEIGHT_DECAY = 1e-4
LABEL_SMOOTH = 0.05
MIXUP_ALPHA = 0.2
MIXUP_PROB = 0.5

BACKBONES = [
    dict(name="tf_efficientnet_b3_ns", img_size=380, drop_rate=0.4, drop_path=0.2, grad_ckpt=False),
    dict(name="vit_base_patch16_384",  img_size=384, drop_rate=0.2, drop_path=0.1, grad_ckpt=True),
]

USE_TEMPERATURE = False
TEMP_T = 1.5

# ----------------------------
# Utils: Safe I/O + Alignment + Normalization + Diagnostics
# ----------------------------
def safe_glob(base_dir, basename):
    cands = glob.glob(os.path.join(base_dir, "**", basename), recursive=True)
    return cands[0] if cands else None

def build_transforms(img_size):
    trn = A.Compose([
        A.Resize(img_size, img_size),
        A.Rotate(limit=10, border_mode=cv2.BORDER_REFLECT_101, p=0.4),
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(0.15, 0.15, p=0.4),
        A.GaussNoise(var_limit=(5, 20), p=0.3),
        A.Normalize(mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ])
    tst = A.Compose([
        A.Resize(img_size, img_size),
        A.Normalize(mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ])
    return trn, tst

def do_mixup(images, targets, alpha=MIXUP_ALPHA):
    if alpha <= 0:
        return images, targets, torch.ones(len(targets), device=targets.device), targets
    lam = np.random.beta(alpha, alpha)
    idx = torch.randperm(images.size(0), device=images.device)
    mixed = lam * images + (1 - lam) * images[idx]
    t1, t2 = targets, targets[idx]
    lam_tensor = torch.full((images.size(0),), lam, device=targets.device, dtype=torch.float32)
    return mixed, (t1, t2), lam_tensor, idx

def mixup_criterion(criterion, preds, targets_tuple, lam_tensor):
    t1, t2 = targets_tuple
    loss1 = criterion(preds, t1)
    loss2 = criterion(preds, t2)
    return (lam_tensor * loss1 + (1 - lam_tensor) * loss2).mean()

def apply_temperature_np(probs, T=1.5):
    logits = np.log(np.clip(probs, 1e-12, 1.0))
    logits /= T
    exps = np.exp(logits - logits.max(axis=1, keepdims=True))
    return exps / exps.sum(axis=1, keepdims=True)

def row_normalize(p):
    p = np.asarray(p, dtype=np.float64)
    p = np.nan_to_num(p, nan=0.0, posinf=0.0, neginf=0.0)
    p = np.clip(p, 1e-12, None)
    s = p.sum(axis=1, keepdims=True)
    s = np.where(s <= 0, 1.0, s)
    return p / s

def check_probs(name, probs):
    p = np.asarray(probs)
    msg = []
    if np.isnan(p).any(): msg.append("NaN")
    if np.isinf(p).any(): msg.append("Inf")
    if p.ndim != 2: msg.append(f"ndim={p.ndim}")
    if p.shape[1] != NUM_CLASSES: msg.append(f"num_classes={p.shape[1]}")
    row_sum = p.sum(axis=1)
    off = np.mean(np.abs(row_sum - 1.0))
    return f"[{name}] shape={p.shape}, rowSumMeanDelta={off:.3e}, flags={'|'.join(msg) if msg else 'OK'}"

def align_by_basename(basenames_ref, df_probs_with_base):
    if isinstance(df_probs_with_base, pd.DataFrame):
        df_probs_with_base = df_probs_with_base
    else:
        raise ValueError("align_by_basename expects DataFrame with 'basename' and prob_* columns")
    df_probs_with_base = df_probs_with_base.set_index("basename").reindex(basenames_ref)
    df_probs_with_base = df_probs_with_base.fillna(1e-12)
    prob_cols = [c for c in df_probs_with_base.columns if c.startswith("prob_")]
    arr = df_probs_with_base[prob_cols].values
    arr = row_normalize(arr)
    return arr

def load_oof_array(path):
    # npz(float32) 우선, npy(float16/32) 호환
    if path.endswith(".npz") and os.path.exists(path):
        z = np.load(path)
        return z["arr_0"].astype(np.float32)
    if path.endswith(".npy") and os.path.exists(path):
        return np.load(path).astype(np.float32)
    raise FileNotFoundError(path)

def save_npz_float32(path, arr):
    arr = np.asarray(arr, dtype=np.float32)
    np.savez_compressed(path, arr)

def corrcoef(a, b):
    a = a.reshape(len(a), -1)
    b = b.reshape(len(b), -1)
    a = a - a.mean(axis=1, keepdims=True)
    b = b - b.mean(axis=1, keepdims=True)
    a_norm = np.sqrt((a**2).sum(axis=1, keepdims=True)) + 1e-12
    b_norm = np.sqrt((b**2).sum(axis=1, keepdims=True)) + 1e-12
    a = a / a_norm; b = b / b_norm
    return float(np.mean(np.sum(a * b, axis=1)))



In [3]:
# ----------------------------
# Dataset
# ----------------------------
class ImageDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = np.array(Image.open(row["filepath"]).convert("RGB"))
        target = int(row["target"])
        if self.transform:
            img = self.transform(image=img)["image"]
        return img, target

class TestDataset(Dataset):
    def __init__(self, filepaths, transform=None):
        self.paths = filepaths; self.transform = transform
    def __len__(self): return len(self.paths)
    def __getitem__(self, idx):
        img = np.array(Image.open(self.paths[idx]).convert("RGB"))
        if self.transform:
            img = self.transform(image=img)["image"]
        return img

# ----------------------------
# Model factory
# ----------------------------
def create_model(model_name, drop_rate=0.2, drop_path=0.1, grad_ckpt=False):
    model = timm.create_model(
        model_name, pretrained=True, num_classes=NUM_CLASSES,
        in_chans=3, drop_rate=drop_rate, drop_path_rate=drop_path
    )
    if grad_ckpt and hasattr(model, "set_grad_checkpointing"):
        model.set_grad_checkpointing(True)
    model = model.to(device)
    model = model.to(memory_format=torch.channels_last)
    return model



In [4]:
# ----------------------------
# Train / Valid (자동 OOM 대처)
# ----------------------------
def train_one_epoch(loader, model, optimizer, loss_fn, scaler=None):
    model.train()
    total_loss, preds_list, targets_list = 0.0, [], []
    for images, targets in tqdm(loader, desc="Train", leave=False):
        images = images.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)
        use_mix = (np.random.rand() < MIXUP_PROB)
        if use_mix:
            images, (t1, t2), lam_tensor, _ = do_mixup(images, targets, alpha=MIXUP_ALPHA)
        optimizer.zero_grad(set_to_none=True)
        try:
            with torch.autocast(device_type="cuda", dtype=torch.float16, enabled=(device.type=="cuda")):
                preds = model(images)
                loss = mixup_criterion(loss_fn, preds, (t1, t2), lam_tensor) if use_mix else loss_fn(preds, targets).mean()
            if scaler and device.type=="cuda":
                scaler.scale(loss).backward(); scaler.step(optimizer); scaler.update()
            else:
                loss.backward(); optimizer.step()
        except RuntimeError as e:
            if "out of memory" in str(e).lower():
                torch.cuda.empty_cache()
                raise e
            else:
                raise e
        total_loss += loss.item()
        preds_list.extend(preds.argmax(1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())
    return {"loss": total_loss/len(loader),
            "acc": accuracy_score(targets_list, preds_list),
            "f1": f1_score(targets_list, preds_list, average="macro")}

@torch.no_grad()
def validate(loader, model, loss_fn):
    model.eval()
    total_loss, preds_list, targets_list = 0.0, [], []
    all_probs = []
    for images, targets in tqdm(loader, desc="Valid", leave=False):
        images = images.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)
        with torch.autocast(device_type="cuda", dtype=torch.float16, enabled=(device.type=="cuda")):
            logits = model(images)
            loss = loss_fn(logits, targets).mean()
            probs = torch.softmax(logits, dim=1).to(torch.float32).cpu().numpy()
        total_loss += loss.item()
        all_probs.append(probs)
        preds_list.extend(probs.argmax(1))
        targets_list.extend(targets.cpu().numpy())
    all_probs = row_normalize(np.vstack(all_probs))
    return {"loss": total_loss/len(loader),
            "acc": accuracy_score(targets_list, preds_list),
            "f1": f1_score(targets_list, preds_list, average="macro"),
            "probs": all_probs, "targets": np.array(targets_list)}

def make_loader(df, transform, batch_size, shuffle, drop_last):
    ds = ImageDataset(df, transform)
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle,
                      num_workers=NUM_WORKERS, pin_memory=True,
                      drop_last=drop_last, persistent_workers=True)

def autotune_batch_size(start_bs, fn_make_loader, try_steps=(1, 0.75, 0.5, 0.25)):
    for scale in try_steps:
        bs = max(1, int(start_bs * scale))
        try:
            loader = fn_make_loader(bs)
            return loader, bs
        except RuntimeError as e:
            if "out of memory" in str(e).lower():
                torch.cuda.empty_cache()
                continue
            else:
                raise e
    raise RuntimeError("Unable to allocate dataloader without OOM.")


In [5]:
# ----------------------------
# Load & join data
# ----------------------------
meta = pd.read_csv(TRAIN_META)
train_csv = pd.read_csv(TRAIN_CSV)
if "basename" not in meta.columns:
    meta["basename"] = meta["filepath"].apply(os.path.basename)
train_csv["basename"] = train_csv["ID"].apply(lambda x: f"{x}.jpg" if not str(x).endswith(".jpg") else x)
df = pd.merge(meta, train_csv[["basename","target"]], on="basename", how="left")
df = df.dropna(subset=["target"]).reset_index(drop=True)
df["target"] = df["target"].astype(int)

# 공통 split 고정
skf = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=SEED)
SPLITS = list(skf.split(df, df["target"]))

# ----------------------------
# Train per backbone (save OOF)
# ----------------------------
all_backbone_fold_scores = {}

for bb in BACKBONES:
    model_name = bb["name"]; img_size = bb["img_size"]
    print(f"\n========== Backbone: {model_name} ({img_size}) ==========")
    trn_tf, tst_tf = build_transforms(img_size)
    fold_scores = []

    for fold, (trn_idx, val_idx) in enumerate(SPLITS):
        print(f"\n----- {model_name} | Fold {fold+1}/{NFOLDS} -----")
        trn_df, val_df = df.iloc[trn_idx].copy(), df.iloc[val_idx].copy()

        # 파일 경로 매핑
        trn_df["filepath"] = trn_df["basename"].apply(lambda b: safe_glob(TRAIN_IMG_DIR, b))
        val_df["filepath"] = val_df["basename"].apply(lambda b: safe_glob(TRAIN_IMG_DIR, b))
        trn_df = trn_df.dropna(subset=["filepath"]).reset_index(drop=True)
        val_df = val_df.dropna(subset=["filepath"]).reset_index(drop=True)

        def _mk_tr_loader(bs):
            return make_loader(trn_df, trn_tf, bs, shuffle=True, drop_last=True)
        def _mk_va_loader(bs):
            return make_loader(val_df, tst_tf, bs, shuffle=False, drop_last=False)

        trn_loader, bs_tr = autotune_batch_size(INIT_BATCH_SIZE, _mk_tr_loader)
        val_loader, bs_va = autotune_batch_size(INIT_BATCH_SIZE, _mk_va_loader)
        print(f"[BS] Train={bs_tr}, Valid={bs_va}")

        model = create_model(model_name, bb["drop_rate"], bb["drop_path"], bb["grad_ckpt"])
        loss_fn = nn.CrossEntropyLoss(label_smoothing=LABEL_SMOOTH, reduction="none")
        optimizer = AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)
        scaler = torch.cuda.amp.GradScaler(enabled=(device.type=="cuda"))

        best_f1 = -1.0; best_val = None
        best_path = f"./models/{model_name}_fold{fold}.pt"
        basenames_val = val_df["basename"].tolist()

        for epoch in range(EPOCHS):
            tr = train_one_epoch(trn_loader, model, optimizer, loss_fn, scaler)
            va = validate(val_loader, model, loss_fn)
            scheduler.step()
            print(f"[{model_name}][Fold {fold}] Ep {epoch+1}/{EPOCHS} | TrF1={tr['f1']:.4f} VaF1={va['f1']:.4f}")
            if va["f1"] > best_f1:
                best_f1 = va["f1"]; best_val = va
                torch.save(model.state_dict(), best_path)
        fold_scores.append(best_f1)

        # OOF 저장: npz float32 + 디버그 리포트
        save_npz_float32(f"./oofs/{model_name}_fold{fold}_probs.npz", best_val["probs"])
        np.save(f"./oofs/{model_name}_fold{fold}_targets.npy", best_val["targets"])
        np.save(f"./oofs/{model_name}_fold{fold}_basenames.npy", np.array(basenames_val))
        with open(f"./oofs/{model_name}_fold{fold}_report.txt", "w") as f:
            f.write(check_probs(f"{model_name}_fold{fold}", best_val["probs"]) + "\n")

        del model; torch.cuda.empty_cache()

    all_backbone_fold_scores[model_name] = fold_scores
    print(f">>> {model_name} fold F1: {fold_scores} | mean={np.mean(fold_scores):.4f}")




========== Backbone: tf_efficientnet_b3_ns (380) ==========

----- tf_efficientnet_b3_ns | Fold 1/5 -----
[BS] Train=16, Valid=16


[tf_efficientnet_b3_ns][Fold 0] Ep 1/30 | TrF1=0.2080 VaF1=0.6819


[tf_efficientnet_b3_ns][Fold 0] Ep 2/30 | TrF1=0.4739 VaF1=0.8211


[tf_efficientnet_b3_ns][Fold 0] Ep 3/30 | TrF1=0.4839 VaF1=0.8728


[tf_efficientnet_b3_ns][Fold 0] Ep 4/30 | TrF1=0.6090 VaF1=0.8690


...

[vit_base_patch16_384][Fold 4] Ep 26/30 | TrF1=0.7820 VaF1=0.8798


[vit_base_patch16_384][Fold 4] Ep 27/30 | TrF1=0.7718 VaF1=0.8819


[vit_base_patch16_384][Fold 4] Ep 28/30 | TrF1=0.8008 VaF1=0.8829


[vit_base_patch16_384][Fold 4] Ep 29/30 | TrF1=0.7433 VaF1=0.8829


[vit_base_patch16_384][Fold 4] Ep 30/30 | TrF1=0.7176 VaF1=0.8821
>>> vit_base_patch16_384 fold F1: [0.9232212960230998, 0.9247294412794339, 0.915197434981356, 0.9061025874975058, 0.8993490249201093] | mean=0.9137


In [6]:
# ----------------------------
# OCR OOF load (optional)
# ----------------------------
def try_load_ocr_oof_for_fold(fold, basenames):
    path = os.path.join(OCR_VALID_PROBS_DIR, f"fold{fold}_ocr_valid.csv")
    if not os.path.exists(path): return None
    df_ocr = pd.read_csv(path)
    prob_cols = [c for c in df_ocr.columns if c.startswith("prob_")]
    df_ocr = df_ocr[["basename"] + prob_cols]
    arr = align_by_basename(basenames, df_ocr)
    return arr

have_ocr_oof = os.path.isdir(OCR_VALID_PROBS_DIR)

# ----------------------------
# Auto-weight grid search (coarse→fine)
# ----------------------------
def eval_weights(eff_list, vit_list, ocr_list, tgt_list, we, wv, wo):
    y_pred, y_true = [], []
    for i, (eff, vit, tgt) in enumerate(zip(eff_list, vit_list, tgt_list)):
        mix = we*eff + wv*vit + (wo*(ocr_list[i]) if ocr_list is not None else 0.0)
        y_pred.append(mix.argmax(1))
        y_true.append(tgt)
    y_pred = np.concatenate(y_pred); y_true = np.concatenate(y_true)
    return f1_score(y_true, y_pred, average="macro")

def grid_search(eff_list, vit_list, ocr_list, tgt_list, step):
    best = (-1.0, 0.5, 0.5, 0.0)
    weights = np.arange(0.0, 1.0+1e-9, step)
    for we in weights:
        for wv in weights:
            rem = 1.0 - we - wv
            if ocr_list is not None:
                if rem < 0: continue
                wo_candidates = np.arange(0.0, rem+1e-9, step)
            else:
                if we + wv <= 0: continue
                wo_candidates = [0.0]
            for wo in wo_candidates:
                s = we+wv+wo; we1, wv1, wo1 = we/s, wv/s, wo/s
                score = eval_weights(eff_list, vit_list, ocr_list, tgt_list, we1, wv1, wo1)
                if score > best[0]: best = (score, we1, wv1, wo1)
    return best

eff_oof, vit_oof, tgt_oof, ocr_oof = [], [], [], [] if have_ocr_oof else None
for fold in range(NFOLDS):
    eff_oof.append(load_oof_array(f"./oofs/tf_efficientnet_b3_ns_fold{fold}_probs.npz"))
    vit_oof.append(load_oof_array(f"./oofs/vit_base_patch16_384_fold{fold}_probs.npz") if os.path.exists(f"./oofs/vit_base_patch16_384_fold{fold}_probs.npz")
                   else load_oof_array(f"./oofs/vit_base_patch16_384_fold{fold}_probs.npy"))
    tgt_oof.append(np.load(f"./oofs/tf_efficientnet_b3_ns_fold{fold}_targets.npy"))
    basenames = np.load(f"./oofs/tf_efficientnet_b3_ns_fold{fold}_basenames.npy", allow_pickle=True).tolist()
    if have_ocr_oof and ocr_oof is not None:
        o = try_load_ocr_oof_for_fold(fold, basenames)
        if o is None: ocr_oof = None
        elif ocr_oof is not None: ocr_oof.append(o)

best = grid_search(eff_oof, vit_oof, ocr_oof, tgt_oof, step=0.1)
# fine search around best
fine = np.linspace(max(0,best[1]-0.1), min(1,best[1]+0.1), 11)
best_f = (-1.0,)*4
for we in fine:
    for wv in np.linspace(max(0,best[2]-0.1), min(1,best[2]+0.1), 11):
        rem = 1.0 - we - wv
        wo_list = [0.0] if ocr_oof is None else np.linspace(max(0,rem-0.1), max(0,rem), 6)
        for wo in wo_list:
            s = we+wv+wo; we1, wv1, wo1 = we/s, wv/s, wo/s
            sc = eval_weights(eff_oof, vit_oof, ocr_oof, tgt_oof, we1, wv1, wo1)
            if sc > best_f[0]: best_f = (sc, we1, wv1, wo1)

best_score, w_eff, w_vit, w_ocr = best_f
print(f"\n[Auto-Weight OOF] best macro F1={best_score:.4f} | w_eff={w_eff:.2f}, w_vit={w_vit:.2f}, w_ocr={(w_ocr if ocr_oof is not None else 0.0):.2f}")
with open("./oofs/best_weights.json", "w") as f:
    json.dump({"score": float(best_score), "w_eff": float(w_eff), "w_vit": float(w_vit), "w_ocr": float(w_ocr if ocr_oof is not None else 0.0)}, f, indent=2)




[Auto-Weight OOF] best macro F1=0.9822 | w_eff=0.35, w_vit=0.16, w_ocr=0.48


In [7]:
# ----------------------------
# Dataset-batched 8-way TTA inference (메모리 친화)
# ----------------------------
sub = pd.read_csv(SUB_CSV)
test_files = []
for name in sub["ID"]:
    if not str(name).endswith(".jpg"): name = f"{name}.jpg"
    cands = glob.glob(os.path.join(TEST_IMG_DIR, "**", name), recursive=True)
    test_files.append(cands[0] if cands else os.path.join(TEST_IMG_DIR, name))

angles = [0, 90, 180, 270]; flips = [False, True]
def rotate_np(img, angle):
    if angle==0: return img
    if angle==90: return cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    if angle==180: return cv2.rotate(img, cv2.ROTATE_180)
    if angle==270: return cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)

backbone_test_probs = {}

for bb in BACKBONES:
    model_name = bb["name"]; img_size = bb["img_size"]
    _, tst_tf = build_transforms(img_size)
    model = create_model(model_name, bb["drop_rate"], bb["drop_path"], bb["grad_ckpt"])
    # fold 평균 가중치
    state_paths = [f"./models/{model_name}_fold{f}.pt" for f in range(NFOLDS)]
    states = [torch.load(p, map_location="cpu") for p in state_paths]
    avg_state = {}
    for k in states[0].keys():
        avg_state[k] = sum(s[k] for s in states) / len(states)
    model.load_state_dict(avg_state, strict=True)
    model.eval()

    def transform_filelist(paths, ang, fl):
        imgs = []
        for p in paths:
            img0 = np.array(Image.open(p).convert("RGB"))
            base = rotate_np(img0, ang)
            img = cv2.flip(base, 1) if fl else base
            imgs.append(img)
        return imgs

    bs_try = INIT_BATCH_SIZE
    test_probs_accum = None
    for ang in angles:
        for fl in flips:
            imgs = transform_filelist(test_files, ang, fl)
            class DummyDS(Dataset):
                def __init__(self, arr, tf): self.arr, self.tf = arr, tf
                def __len__(self): return len(self.arr)
                def __getitem__(self, i): return self.tf(image=self.arr[i])["image"]
            ds = DummyDS(imgs, tst_tf)

            def _mk(bs):
                return DataLoader(ds, batch_size=bs, shuffle=False,
                                  num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=False)

            loader = None
            for scale in (1, 0.75, 0.5, 0.25):
                try:
                    bs = max(1, int(bs_try*scale))
                    loader = _mk(bs)
                    it = iter(loader); xb = next(it)
                    xb = xb.to(device, non_blocking=True)
                    with torch.no_grad(), torch.autocast(device_type="cuda", dtype=torch.float16, enabled=(device.type=="cuda")):
                        _ = model(xb)
                    break
                except Exception as e:
                    if "out of memory" in str(e).lower():
                        torch.cuda.empty_cache(); continue
                    else:
                        raise e
            if loader is None:
                raise RuntimeError("TTA loader OOM")

            probs_run = []
            for xb in tqdm(loader, desc=f"TTA {model_name} ang{ang} fl{fl}"):
                xb = xb.to(device, non_blocking=True)
                with torch.no_grad(), torch.autocast(device_type="cuda", dtype=torch.float16, enabled=(device.type=="cuda")):
                    logits = model(xb)
                    probs = torch.softmax(logits, dim=1).to(torch.float32).cpu().numpy()
                probs_run.append(probs)
            probs_run = row_normalize(np.vstack(probs_run))

            if test_probs_accum is None:
                test_probs_accum = probs_run
            else:
                test_probs_accum += probs_run
            torch.cuda.empty_cache()

    test_prob = (test_probs_accum / 8.0).astype(np.float32)
    if USE_TEMPERATURE:
        test_prob = apply_temperature_np(test_prob, T=TEMP_T)
    backbone_test_probs[model_name] = row_normalize(test_prob)
    save_npz_float32(f"./test_probs/{model_name}_test_probs.npz", backbone_test_probs[model_name])
    del model; torch.cuda.empty_cache()



TTA vit_base_patch16_384 ang270 flTrue: 100%|████████████████████████████████████████████████████████████████████████████████| 197/197 [00:09<00:00, 19.96it/s]


In [8]:
# ----------------------------
# Optional OCR test probs load (+정렬/정규화)
# ----------------------------
ocr_test_probs = None
if os.path.exists(OCR_TEST_PROBS_CSV):
    ocr_df = pd.read_csv(OCR_TEST_PROBS_CSV)
    prob_cols = [c for c in ocr_df.columns if c.startswith("prob_")]
    sub_ids = pd.read_csv(SUB_CSV)["ID"].astype(str).apply(lambda x: x if x.endswith(".jpg") else f"{x}.jpg")
    df_tmp = ocr_df.set_index("ID").reindex(sub_ids).reset_index().rename(columns={"index":"ID"})
    df_tmp = df_tmp.fillna(1e-12)
    ocr_test_probs = row_normalize(df_tmp[prob_cols].values.astype(np.float32))
    print(check_probs("OCR_test", ocr_test_probs))
else:
    print("No OCR test probs. Skip OCR in final ensemble.")

# ----------------------------
# Final weighted ensemble & save (with expected F1 & diagnostics)
# ----------------------------
eff_probs = np.load("./test_probs/tf_efficientnet_b3_ns_test_probs.npz")["arr_0"]
vit_probs = np.load("./test_probs/vit_base_patch16_384_test_probs.npz")["arr_0"]

# 가중치 정규화
ws = w_eff + w_vit + (w_ocr if ocr_test_probs is not None else 0.0)
w_eff, w_vit, w_ocr = w_eff/ws, w_vit/ws, (w_ocr/ws if ocr_test_probs is not None else 0.0)

# (1) OOF 기반 예상 F1
print("\n[Step] Evaluating expected F1 using OOF probs...")
eff_oof2, vit_oof2, tgt_oof2, ocr_oof2 = [], [], [], [] if have_ocr_oof else None
for fold in range(NFOLDS):
    eff = load_oof_array(f"./oofs/tf_efficientnet_b3_ns_fold{fold}_probs.npz")
    vit = load_oof_array(f"./oofs/vit_base_patch16_384_fold{fold}_probs.npz") if os.path.exists(f"./oofs/vit_base_patch16_384_fold{fold}_probs.npz") \
          else load_oof_array(f"./oofs/vit_base_patch16_384_fold{fold}_probs.npy")
    tgt = np.load(f"./oofs/tf_efficientnet_b3_ns_fold{fold}_targets.npy")
    basenames = np.load(f"./oofs/tf_efficientnet_b3_ns_fold{fold}_basenames.npy", allow_pickle=True).tolist()
    eff_oof2.append(row_normalize(eff))
    vit_oof2.append(row_normalize(vit))
    tgt_oof2.append(tgt)
    if have_ocr_oof and ocr_oof2 is not None:
        o = try_load_ocr_oof_for_fold(fold, basenames)
        if o is None: ocr_oof2 = None
        elif ocr_oof2 is not None: ocr_oof2.append(row_normalize(o))

y_pred, y_true = [], []
for i, (eff, vit, tgt) in enumerate(zip(eff_oof2, vit_oof2, tgt_oof2)):
    mix = w_eff*eff + w_vit*vit + (w_ocr*ocr_oof2[i] if ocr_oof2 is not None else 0.0)
    y_pred.append(mix.argmax(1))
    y_true.append(tgt)
y_pred = np.concatenate(y_pred)
y_true = np.concatenate(y_true)
exp_f1 = f1_score(y_true, y_pred, average="macro")

# (2) Test 예측
final_probs = w_eff*eff_probs + w_vit*vit_probs + (w_ocr*ocr_test_probs if ocr_test_probs is not None else 0.0)
final_probs = row_normalize(final_probs)
final_preds = np.argmax(final_probs, axis=1)

# (3) 진단 리포트
with open("./debug/ensemble_diagnostics.txt", "w") as f:
    f.write(check_probs("Eff_test", eff_probs) + "\n")
    f.write(check_probs("ViT_test", vit_probs) + "\n")
    if ocr_test_probs is not None:
        f.write(check_probs("OCR_test", ocr_test_probs) + "\n")
    f.write(f"Pairwise corr (test probs):\n")
    f.write(f"  Eff-ViT: {corrcoef(eff_probs, vit_probs):.4f}\n")
    if ocr_test_probs is not None:
        f.write(f"  Eff-OCR: {corrcoef(eff_probs, ocr_test_probs):.4f}\n")
        f.write(f"  ViT-OCR: {corrcoef(vit_probs, ocr_test_probs):.4f}\n")
    f.write(f"\nWeights (normalized): eff={w_eff:.3f}, vit={w_vit:.3f}, ocr={(w_ocr if ocr_test_probs is not None else 0.0):.3f}\n")
    f.write(f"Expected OOF F1: {exp_f1:.4f}\n")



[OCR_test] shape=(3140, 17), rowSumMeanDelta=4.176e-17, flags=OK

[Step] Evaluating expected F1 using OOF probs...


In [9]:
# ============================================================
# (4) Alignment check (정렬 검증)
# ============================================================
align_log_path = "./debug/ensemble_alignment_check.txt"
sub_ids = pd.read_csv(SUB_CSV)["ID"].astype(str).apply(lambda x: x if x.endswith(".jpg") else f"{x}.jpg")

with open(align_log_path, "w") as f:
    f.write("==== Alignment Check Report ====\n")
    f.write(f"Total test samples: {len(sub_ids)}\n\n")

    # 각 모델의 test 파일명 정렬 상태 비교
    for model_name in ["tf_efficientnet_b3_ns", "vit_base_patch16_384"]:
        test_prob_path = f"./test_probs/{model_name}_test_probs.npz"
        if not os.path.exists(test_prob_path):
            f.write(f"[{model_name}] ❌ Missing probs file\n")
            continue
        # 각 모델이 처리한 파일 순서 확인용 캐시 (test_files를 재활용)
        f.write(f"[{model_name}] {test_prob_path} found.\n")
        # 실제 파일 경로에서 basename 추출
        model_files = [os.path.basename(x) for x in test_files]
        diff = [i for i, (a, b) in enumerate(zip(sub_ids, model_files)) if a != b]
        if len(diff) == 0:
            f.write(f"✅ Order match with sample_submission ({len(sub_ids)} items)\n")
        else:
            f.write(f"⚠️ Mismatch in {len(diff)} indices (showing first 5): {diff[:5]}\n")
        f.write("\n")

    # OCR 정렬 확인
    if ocr_test_probs is not None and os.path.exists(OCR_TEST_PROBS_CSV):
        ocr_df = pd.read_csv(OCR_TEST_PROBS_CSV)
        ocr_ids = ocr_df["ID"].astype(str).apply(lambda x: x if x.endswith(".jpg") else f"{x}.jpg")
        missing = set(sub_ids) - set(ocr_ids)
        extra = set(ocr_ids) - set(sub_ids)
        if len(missing) == 0 and len(extra) == 0:
            f.write("✅ OCR alignment OK (IDs fully matched)\n")
        else:
            f.write(f"⚠️ OCR alignment mismatch: missing={len(missing)}, extra={len(extra)}\n")
    else:
        f.write("ℹ️ No OCR probs to check alignment\n")

print(f"🧭 Alignment report saved → {align_log_path}")



🧭 Alignment report saved → ./debug/ensemble_alignment_check.txt


In [12]:

# 제출 파일
sub = pd.read_csv(SUB_CSV)
sub["target"] = final_preds
mean_eff = np.mean(all_backbone_fold_scores["tf_efficientnet_b3_ns"])
mean_vit = np.mean(all_backbone_fold_scores["vit_base_patch16_384"])
out_name = f"sub_v8safe_eff{mean_eff:.4f}_vit{mean_vit:.4f}_w{w_eff:.2f}-{w_vit:.2f}-{(w_ocr if ocr_test_probs is not None else 0.0):.2f}_expF1{exp_f1:.4f}.csv"
sub.to_csv(out_name, index=False)

print(f"\n✅ Saved submission with expected F1={exp_f1:.4f}")
print(f"📁 {out_name}")
print("🧪 Diagnostics → ./debug/ensemble_diagnostics.txt")



✅ Saved submission with expected F1=0.9822
📁 sub_v8safe_eff0.9394_vit0.9137_w0.35-0.16-0.48_expF10.9822.csv
🧪 Diagnostics → ./debug/ensemble_diagnostics.txt
